In [1]:
!pip install -r requirements.txt

  Using cached langchain-0.0.225-py3-none-any.whl.metadata (13 kB)
Using cached langchain-0.0.225-py3-none-any.whl (1.2 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.14
    Uninstalling langchain-0.1.14:
      Successfully uninstalled langchain-0.1.14


In [2]:
%pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
PINECONE_API_ENV = os.environ.get("PINECONE_API_ENV")

In [5]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data, 
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [6]:
extracted_data = load_pdf("data/")

In [7]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [9]:
# Download Embedding Model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:
embeddings = download_hugging_face_embeddings()

In [11]:
index_name = "medical-chatbot"

In [12]:
# Upsert the data to Pinecone
# docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [13]:
docsearch_query = PineconeVectorStore.from_existing_index(index_name, embeddings)
query = "What are Allergies"

docs = docsearch_query.similarity_search(query, k=3)
print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly ca

In [14]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [15]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [16]:
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin", 
                  model_type="llama", 
                  config={'max_new_tokens': 512, 'temperature': 0.8})

In [17]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=docsearch_query.as_retriever(search_kwargs={'k': 2}),
                                 return_source_documents=True,
                                 chain_type_kwargs=chain_type_kwargs
                                 )

In [20]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

Response :  Fever is a symptom of an underlying illness or infection. It is characterized by an elevated body temperature, usually above 100°F (37.8°C), and can be accompanied by other symptoms such as shivering, sweating, headache, fatigue, and muscle aches. The exact cause of fever depends on the underlying condition or infection that is causing it, but some common causes include viral or bacterial infections, autoimmune disorders, and allergic reactions. Treatment for fever usually involves addressing the underlying cause and managing symptoms with medication or other interventions.
